<a href="https://colab.research.google.com/github/Karthick47v2/mcq-generator/blob/main/summarize-extract-genQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TODO : OPTIMIZEEEEEE
# TODO : RUN MULTI TEXT

In [ ]:
# CREATE BATCH OF TEXTS

In [ ]:
# DONT FORGET TO CHANGE TO CUDA VERSION
# !pip3 install torch==1.8.2+cpu torchvision==0.9.2+cpu torchaudio==0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
!pip3 install transformers==4.15.0    # only this ver works with FastT5
!pip3 install SentencePiece
!pip3 install git+https://github.com/boudinfl/pke.git
# FAST T5
!pip3 install fastt5

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import pke
import string
import re
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# from fastT5 import export_and_get_onnx_model, generate_onnx_representation, quantize
# from transformers import T5Config, AutoTokenizer

# t = 't5-base'

# path = generate_onnx_representation(t)
# q_path = quantize(path)
# tokenizer_onnx = AutoTokenizer.from_pretrained(t)
# conf = T5Config.from_pretrained(t)

In [7]:
# tokenizer_onnx.save_pretrained('models/')
# conf.save_pretrained('models/')

In [10]:
# !cp -r models '/content/gdrive/MyDrive/mcq-gen/t5-summarize'

In [11]:
from fastT5 import get_onnx_model, get_onnx_runtime_sessions, OnnxT5 
from transformers import AutoTokenizer
from pathlib import Path
import os

# summarize
model_path = '/content/gdrive/MyDrive/mcq-gen/t5-summarize/t5-base'
model_name = "t5-base"
encoder_path = os.path.join(model_path, f"{model_name}-encoder-quantized.onnx")
decoder_path = os.path.join(model_path, f"{model_name}-decoder-quantized.onnx")
init_decoder_path = os.path.join(model_path, f"{model_name}-init-decoder-quantized.onnx")

model_sessions = get_onnx_runtime_sessions((encoder_path,decoder_path,init_decoder_path))
sum_model = OnnxT5(model_path, model_sessions)
sum_tokenizer = AutoTokenizer.from_pretrained(model_path)

# que model
model_path = '/content/gdrive/MyDrive/mcq-gen/t5-question'
model_name = 't5_squad_v1'
encoder_path = os.path.join(model_path, f"{model_name}-encoder-quantized.onnx")
decoder_path = os.path.join(model_path, f"{model_name}-decoder-quantized.onnx")
init_decoder_path = os.path.join(model_path, f"{model_name}-init-decoder-quantized.onnx")

model_sessions = get_onnx_runtime_sessions((encoder_path,decoder_path,init_decoder_path))
q_model = OnnxT5(model_path, model_sessions)
q_tokenizer = AutoTokenizer.from_pretrained(model_path)

In [12]:
def preprocess_splitted_text(text):
  # https://huggingface.co/docs/transformers/preprocessing  - Tokenize
  encode = sum_tokenizer.encode_plus("summarize: " + text, return_tensors='pt', max_length=512, pad_to_max_length=False, truncation=True)
  return encode["input_ids"], encode["attention_mask"]

In [13]:
def preprocess_bulk_text(text):
  text = text.strip()
  text = re.sub('[\u2010-\u2013]', '-', text)
  text = re.sub('[^a-zA-Z0-9\.,-?]', ' ', text)
  text = re.sub(' {2,}', ' ', text)
  return text

In [14]:
def split_text(text, range=300):
  bulk_text = preprocess_bulk_text(text)
  splitted_texts = []
  while(len(bulk_text) > range):
    i = range
    while((i < len(bulk_text)) and (bulk_text[i] != '.')):
      i += 1
    splitted_texts.append(bulk_text[:(i+1)])
    bulk_text = bulk_text.replace(bulk_text[:(i+1)], "")

  return splitted_texts

In [15]:
def postprocess_summary(text):
  output = ""

  for x in sent_tokenize(text):
    x = x.capitalize()
    output += " " + x

  return output

In [16]:
def summarize(text):
  input_tokens_ids, attention_mask = preprocess_splitted_text(text)
  summary_encoded = sum_model.generate(input_ids=input_tokens_ids, 
                                   attention_mask=attention_mask,
                                   num_beams=3,
                                   num_return_sequences=1,
                                   no_repeat_ngram_size=2, 
                                   max_length=300, 
                                   early_stopping=True)
  # decode summarized token
  output = sum_tokenizer.decode(summary_encoded[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

  return postprocess_summary(output)

In [17]:
def extract_keywords(text, kw_pop):
  ex = pke.unsupervised.MultipartiteRank()
  ex.load_document(text)   

  pos = {'PROPN','NOUN'}
  stoplist = list(string.punctuation) 
  stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
  stoplist += stopwords.words('english')
  ex.candidate_selection(pos=pos, stoplist=stoplist)

  ex.candidate_weighting(alpha=1.1, threshold=0.75, method='average')

  kw = ex.get_n_best(n=kw_pop)

  kw_ls = []

  for i in kw:
    kw_ls.append(i[0])

  return kw_ls

In [18]:
def filter_keywords(original, summarized, kw_pop=5):
  orig_ls = extract_keywords(original, kw_pop)
  sum_ls = extract_keywords(summarized, kw_pop)

  orig_ls = set(orig_ls)
  return list(orig_ls.intersection(sum_ls))

In [19]:
def preprocess_summary(context, answer):
  text = "context: {} answer: {}".format(context, answer)
  # https://huggingface.co/docs/transformers/preprocessing  - Tokenize
  encode = q_tokenizer.encode_plus(text, return_tensors='pt', max_length = 382, pad_to_max_length=False, truncation=True)
  return encode["input_ids"], encode["attention_mask"]

In [20]:
def gen_question(context, answer):
  input_tokens_ids, attention_mask = preprocess_summary(context, answer)

  question_encoded = q_model.generate(input_ids=input_tokens_ids, 
                                             attention_mask=attention_mask,
                                             num_beams=5,
                                             no_repeat_ngram_size=2, 
                                             max_length=72, 
                                             early_stopping=True)
  # decode summarized token
  output = q_tokenizer.decode(question_encoded[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
  output = output.replace("question: ", "")
  output = output.strip()
  return output

In [21]:
full_text = """
Centralized DBMS architecture, It means files are stored in a single location and it is maintained from only that location. It is usually used by universities, companies and banks.
# Advantages of Centralized DBMS architecture, 
# The data integrity is increased. So, the data is accurate and consistent.
# Since all data in one place, data security is increased.
# It is cheaper as it requires less power and maintenance.
# Disadvantages of Centralized DBMS architecture,
# Lot of data access traffic which may bottleneck in some situations.
# If multiple users try to access it simultaneously this may reduce the efficiency of the system.
# There is no database recovery measure when system failure occurs, so all data will be lost.
# Since all data is centralized, it takes more time to access it and search. And if the network is slow, then it takes even more time.
"""

In [22]:
splitted_text = split_text(full_text)
summary = []
filtered_kw = []

for i in range(len(splitted_text)):
  summary.append(summarize(splitted_text[i]))
  filtered_kw.append(filter_keywords(splitted_text[i], summary[i]))

for i in range(len(filtered_kw)):
  for x in filtered_kw[i]:
    print(gen_question(splitted_text[i], x))
    print(x)

Along with universities and banks, what organizations use Centralized DBMS?
companies
Along with universities and companies, what other institutions use Centralized DBMS?
banks
What is increased with Centralized DBMS?
data integrity
If multiple users try to access it simultaneously this may reduce the efficiency of what?
system
